In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
data = pd.read_csv('C:/Users/Home/Documents/Coursera/final_statement/features.csv', index_col='match_id')

In [ ]:
X = data.drop(['duration',
               'radiant_win',
               'tower_status_radiant',
               'tower_status_dire',
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1)

In [ ]:
for i in list(X):
    if X[i].isnull().sum() > 0:
        print(i, ' - ', X[i].isnull().sum())

#### Пропуски имеют:
- игровое время первой крови
- команда, совершившая первую кровь
- игрок, причастный к событию
- второй игрок, причастный к событию
        видимо команды играли слишком осторожно и никто не был убит в первые 5 минут
- время первого приобретения командой Radiant предмета "bottle"
- время приобретения командой Radiant предмета "courier"
- время приобретения командой Radiant предмета "flying_courier"
- время установки командой Radiant первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- время первого приобретения командой Dire предмета "bottle"
- время приобретения командой Dire предмета "courier"
- время приобретения командой Dire предмета "flying_courier"
- время установки командой Dire первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
        судя по всему команды не приобрели нужные предметы и не выставляли наблюдателей

In [ ]:
X = X.fillna(0)

In [ ]:
y = data['radiant_win']

#### Столбец, содержащий целевую переменную: radiant_win

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
nambers = [10, 20, 30, 40]
for i in nambers:
    clf = GradientBoostingClassifier(n_estimators=i, random_state=1)
    start = datetime.now()
    score = cross_val_score(clf, X, y, scoring='roc_auc', cv=kf)
    print("Namber of trees: %i, ROC_AUC: %.1f, time: %s" % (i, score.mean() * 100, datetime.now() - start))

#### Кросс-валидация для градиентного бустинга с 30 деревьями проводилась 1 минуту 47 секунд.

#### Имеет смысл использовать больше 30 деревьев в градиентном бустинге, т.к. качество продолжает расти при дальнейшем увеличении их числа.
Чтобы ускорить обучение можно уменьшить глубину деревьев или использовать для кросс-валидации не всю выборку.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def score_roc_auc (data):
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    C = [1e-7, 1e-4, 1e-3, 1]
    for i in C:
        logit = LogisticRegression(C=i, random_state=1, penalty='l2')
        start = datetime.now()
        score = cross_val_score(logit, data, y, scoring='roc_auc', cv=kf)
        print("C: %f, ROC_AUC: %.1f, time: %s" % (i, score.mean() * 100, datetime.now() - start))

In [ ]:
score_roc_auc(X_scaled)

#### Качество у логистической регрессии над всеми исходными признаками получилось 71.6
Оно выше, чем у градиентного бустинга, логистическая регрессия по сравнению с градиентным бустингом работает быстре.

In [ ]:
X_clean = X.drop(['lobby_type',
                  'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero',
                  'first_blood_player1', 'first_blood_player2'], axis=1)

In [ ]:
X_clean_scaled = scaler.fit_transform(X_clean)

In [ ]:
score_roc_auc(X_clean_scaled)

#### Удаление категориальных признаков не повлияло на качество (значение метрики 71.6)

In [ ]:
hero = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
uniq = np.array([])
for i in hero:
    uniq = np.append(uniq, pd.unique(X[i]))
N = np.shape(pd.value_counts(uniq))[0]
print(N)

#### 108 различных идентификаторов героев существует в данной игре

In [ ]:
hero_c = [c for c in X.columns if 'hero' in c]
all_heroes_id = np.unique(X[hero_c])
wb = {}
for id in all_heroes_id:
    # Мы используем + 0 для автоматического приведения bool-->int.
    r = [(X['r%d_hero' % n] == id) + 0 for n in range(1, 6)]
    d = [(X['d%d_hero' % n] == id) + 0 for n in range(1, 6)]
    wb['hero%s' % id] = sum(r) - sum(d)
X_pick = X_clean.assign(**wb)

In [ ]:
X_pick_scaled = scaler.fit_transform(X_pick)

In [ ]:
score_roc_auc(X_pick_scaled)

#### Качество при добавлении "мешка слов" по героям улучшилось и стало равным 75.2, т.к. мы стали использовать более полную информацию о игроках.

In [ ]:
X_test = pd.read_csv('features_test.csv', index_col='match_id')

In [ ]:
C = 1e-3
logit = LogisticRegression(C=C, random_state=1, penalty='l2')
logit.fit(X_pick_scaled, y)

In [ ]:
X_test = X_test.fillna(0)

In [ ]:
X_test_clean = X_test.drop(['lobby_type',
                  'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero',
                  'first_blood_player1', 'first_blood_player2'], axis=1)

In [ ]:
hero_c = [c for c in X_test.columns if 'hero' in c]
all_heroes_id = np.unique(X_test[hero_c])
wb = {}
for id in all_heroes_id:
    # Мы используем + 0 для автоматического приведения bool-->int.
    r = [(X_test['r%d_hero' % n] == id) + 0 for n in range(1, 6)]
    d = [(X_test['d%d_hero' % n] == id) + 0 for n in range(1, 6)]
    wb['hero%s' % id] = sum(r) - sum(d)
X_test_pick = X_test_clean.assign(**wb)

In [ ]:
X_test_pick_scaled = scaler.transform(X_test_pick)

In [ ]:
pred = logit.predict_proba(X_test_pick_scaled)[:, 1]

In [ ]:
pred.max()

In [ ]:
pred.min()

In [ ]:
X_test_pick.index.dtype

#### У лучшего из алгоритмов получилось:
- 0.0096 минимальное значение прогноза
- 0.9947 максимальное значение прогноза

In [ ]:
answer = pd.DataFrame(data=[X_test_pick.index, pred], index=['match_id', 'radiant_win']).T

In [ ]:
answer['match_id'] = answer['match_id'].astype('Int32')

In [ ]:
answer.dtypes

In [ ]:
answer.to_csv('submission.csv', index=False)